# General

In [6]:
# pd.set_option('display.max_colwidth', 1000)

In [7]:
# !pip install nltk

In [8]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

# Kaggle Dataset

In [9]:
import pandas as pd

df = pd.read_csv('./data/artist.csv')
print(df.shape)
df.head()

(2252446, 5)


,ArtistId,mbid,artist,AreaId,tag
0,1,89ad4ac3-39f7-470e-963a-56509c546377,Various Artists,NaN,rock; trance; electronic; classical; pop; spec...
1,2,c06aa285-520e-40c0-b776-83d2c9e8a6d1,Deleted Artist,NaN,NaN
2,4,10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8,Massive Attack,3821.0,trip-hop; rock; electronic; downtempo; alterna...
3,6,ea738cc5-5b1a-44a4-94ab-ed0c1bd71ecf,Apartment 26,221.0,electronic; jazz; metal; british; uk; nu metal...
4,7,025d1400-5420-4fc8-a482-2b59096fd22c,Dr. Evil,NaN,bogus artist; fictitious artist; fictitious-ar...


In [10]:
allTagList = []

def clean(tags):
    tagList = tags.split(" ")
    tagSet = set()
    for tag in tagList:
        if tag != "" and len(tag) >= 3 and len(tag) <= 20:
            tagSet.add(tag)
            allTagList.append(tag)
    
    return " ".join(tagSet)

# get non-ambiguous artists
df2 = df[df['ambiguous_artist'] == False]

# get artist name and tags only
df2 = df2[['tags_mb', 'artist_mb']]

# remove duplicate artists
df2['artist_mb_duplicate'] = df2.loc[:, 'artist_mb']
df2['artist_mb_duplicate'] = df2['artist_mb_duplicate'].str.lower()
df2 = df2.drop_duplicates(subset=['artist_mb_duplicate'])


df2['tags_mb'] = df2['tags_mb'].str.lower()
df2 = df2.dropna(subset=['tags_mb'])
df2 = df2.drop_duplicates(subset=['artist_mb'])
df2 = df2[df2['tags_mb'].apply(lambda x: isinstance(x, str))]

# make each tag a single word
df2['tags_mb'] = df2['tags_mb'].str.replace(' ', '')
df2['tags_mb'] = df2['tags_mb'].str.replace(';', ' ')

# remove special characters
df2['tags_mb'] = df2['tags_mb'].str.replace(
    r'[^a-z0-9\s]',
    '',
    regex=True,
)

# remove tag duplicates and clean tags
df2['tags_mb'] = df2['tags_mb'].apply(clean)

allTagList.sort()
print(allTagList)

print(df2.shape)
df2.head()

KeyError: 'ambiguous_artist'

In [ ]:
df2 = df2[:50000]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df2['tags_mb'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [7]:
df3 = pd.DataFrame(similarities, columns=df2['artist_mb'], index=df2['artist_mb']).reset_index()

df3.head()

artist_mb,artist_mb,Coldplay,Radiohead,Red Hot Chili Peppers,Rihanna,Eminem,The Killers,Kanye West,Nirvana,Muse,...,Irrsterne,Irwin Gage,Julián Aguirre,La Gente de Omar Geles,Les Apaches,MASTERLINK,Mörder Machine,Out of Darkness,Rat Scabies,Richard Williams
0,Coldplay,1.000000,0.305068,0.068752,0.018188,0.000000,0.115676,0.028909,0.061714,0.215478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Radiohead,0.305068,1.000000,0.032766,0.000000,0.000000,0.061752,0.000000,0.097302,0.171929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111955,0.0
2,Red Hot Chili Peppers,0.068752,0.032766,1.000000,0.034782,0.089128,0.167616,0.027038,0.150034,0.164141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Rihanna,0.018188,0.000000,0.034782,1.000000,0.047933,0.000000,0.100217,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,Eminem,0.000000,0.000000,0.089128,0.047933,1.000000,0.025704,0.277201,0.017342,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [8]:
input_artist = "Coldplay"
recommendations = pd.DataFrame(df3.nlargest(11,input_artist)['artist_mb'])
recommendations = recommendations[recommendations['artist_mb']!=input_artist]
print(recommendations)

                          artist_mb
98                            Keane
912                      Starsailor
1403  The Good, the Bad & the Queen
738                 Vanessa Carlton
723                       Ben Folds
3526                  Jarvis Cocker
71                          Placebo
3578                      Tom McRae
18                            Oasis
210                           Train


# Scraped Musicbrainz 2023 Dataset

In [42]:
import pandas as pd

df_mb = pd.read_csv('./data/mb_export.csv')
print(df_mb.shape)
df_mb.head()

(2252446, 5)


,ArtistId,mbid,artist,AreaId,tag
0,1,89ad4ac3-39f7-470e-963a-56509c546377,Various Artists,NaN,rock; trance; electronic; classical; pop; spec...
1,2,c06aa285-520e-40c0-b776-83d2c9e8a6d1,Deleted Artist,NaN,NaN
2,4,10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8,Massive Attack,3821.0,trip-hop; rock; electronic; downtempo; alterna...
3,6,ea738cc5-5b1a-44a4-94ab-ed0c1bd71ecf,Apartment 26,221.0,electronic; jazz; metal; british; uk; nu metal...
4,7,025d1400-5420-4fc8-a482-2b59096fd22c,Dr. Evil,NaN,bogus artist; fictitious artist; fictitious-ar...


In [43]:
df_mb2 = df_mb.dropna()
df_mb2.shape

(155500, 5)

In [44]:
df_mb2 = df_mb2[['artist', 'tag']]

allTagList = []

def clean(tags):
    tagList = tags.split(" ")
    tagSet = set()
    for tag in tagList:
        if tag != "" and len(tag) >= 3 and len(tag) <= 20:
            tagSet.add(tag)
            allTagList.append(tag)
    
    return " ".join(tagSet)

# remove duplicate artists
df_mb2['artist'] = df_mb2['artist'].str.lower()
df_mb2 = df_mb2.drop_duplicates(subset=['artist'])

# make each tag a single word
df_mb2['tag'] = df_mb2['tag'].str.replace(' ', '')
df_mb2['tag'] = df_mb2['tag'].str.replace(';', ' ')

# remove special characters
df_mb2['tag'] = df_mb2['tag'].str.replace(
    r'[^a-z0-9\s]',
    '',
    regex=True,
)

# remove tag duplicates and clean tags
df_mb2['tag'] = df_mb2['tag'].apply(clean)

print(df_mb2.shape)
df_mb2.head()


(148783, 2)


,artist,tag
2,massive attack,downtempo chillout english britannique oldesta...
3,apartment 26,jazz english jazzmetal british electronic meta...
5,robert miles,downtempo dreamtrance nujazz trance electronic...
6,vincent gallo,warp american
7,squirrel nut zippers,jazz swing american swingrevival


In [45]:
df_mb2 = df_mb2[:50000]

In [46]:
df_mb2['split_tags'] = df_mb2['tag'].str.split(" ", expand=False)

df_mb2.head()

,artist,tag,split_tags
2,massive attack,downtempo chillout english britannique oldesta...,"[downtempo, chillout, english, britannique, ol..."
3,apartment 26,jazz english jazzmetal british electronic meta...,"[jazz, english, jazzmetal, british, electronic..."
5,robert miles,downtempo dreamtrance nujazz trance electronic...,"[downtempo, dreamtrance, nujazz, trance, elect..."
6,vincent gallo,warp american,"[warp, american]"
7,squirrel nut zippers,jazz swing american swingrevival,"[jazz, swing, american, swingrevival]"


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df_mb2['tag'])

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [82]:
import numpy as np

# Create an empty DataFrame to store the top 10 similar artists for each artist
top_similar_artists_df = pd.DataFrame(index=df_mb2['artist'], columns=[f"SimilarArtist_{i+1}" for i in range(10)])

counter = 0

for index, row in df_mb2.iterrows():
    artist_name = row['artist']
    artist_tags = row['tag']
    vectorised_artist_tags = vectorizer.transform([artist_tags])
    similarities = cosine_similarity(vectorized, vectorised_artist_tags)
    similar_indices = np.argsort(similarities.flatten())[:-12:-1]
    similar_artists = df_mb2['artist'].iloc[similar_indices].tolist()[1:]
    top_similar_artists_df.loc[artist_name] = similar_artists

    counter += 1
    if counter == 5:
        break

top_similar_artists_df.head()

,SimilarArtist_1,SimilarArtist_2,SimilarArtist_3,SimilarArtist_4,SimilarArtist_5,SimilarArtist_6,SimilarArtist_7,SimilarArtist_8,SimilarArtist_9,SimilarArtist_10
artist,,,,,,,,,,
massive attack,morcheeba,portishead,chris coco,terranova,leftfield,unkle,alpha,celestial,helicopter girl,solar fields
apartment 26,shining,exilia,bloodsimple,emil bulls,flaw,slitheryn,hamlet,static‐x,jonathan davis,tracktor bowling
robert miles,dzihan & kamien,terranova,kevin yost,trentemøller,alpha,goldfish,st germain,bt,john digweed,torsten stenzel
vincent gallo,kelli hand,beans,kenny larkin,dj mink,drexciya,home video,thk,elecktroids,ko-wreck technique,rubber johnny
squirrel nut zippers,eddie sauter,john pizzarelli,tyree glenn,dave mckenna,charlie barnet,cozy cole,dick mcdonough,lester young,charlie shavers,eric delaney


In [17]:
df_mb3 = pd.DataFrame(similarities, columns=df_mb2['artist'], index=df_mb2['artist']).reset_index()

df_mb3.head()

artist,artist,massive attack,apartment 26,robert miles,vincent gallo,squirrel nut zippers,giant sand,éric serra,william s. burroughs,bob dylan,...,ayşe tütüncü,mo.do.,depraved,the smartpils,freak electric,carl engel,ibis,d numbers,黃國倫,陳少琪
0,massive attack,1.000000,0.130708,0.238437,0.000000,0.000000,0.052470,0.000000,0.000000,0.009024,...,0.000000,0.0,0.085286,0.078939,0.0,0.000000,0.0,0.0,0.0,0.0
1,apartment 26,0.130708,1.000000,0.049098,0.000000,0.052091,0.000000,0.000000,0.000000,0.000000,...,0.202839,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,robert miles,0.238437,0.049098,1.000000,0.000000,0.000000,0.000000,0.116275,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,vincent gallo,0.000000,0.000000,0.000000,1.000000,0.185875,0.171455,0.000000,0.225369,0.059863,...,0.000000,0.0,0.000000,0.000000,0.0,0.349224,0.0,0.0,0.0,0.0
4,squirrel nut zippers,0.000000,0.052091,0.000000,0.185875,1.000000,0.112256,0.000000,0.147555,0.039194,...,0.256812,0.0,0.000000,0.000000,0.0,0.228646,0.0,0.0,0.0,0.0


In [21]:
import numpy as np

# Create an empty DataFrame to store the top 10 similar artists for each artist
top_similar_df = pd.DataFrame(index=df_mb2['artist'], columns=[f"SimilarArtist_{i+1}" for i in range(10)])

# Loop through each row in the similarities matrix and get the names of top 10 similar artists
for i, artist in enumerate(df_mb2['artist']):
    # Exclude self-similarity and get top 10 similar artists
    similar_indices = np.argsort(similarities[i])[::-1][1:11]
    similar_artists = df_mb2['artist'].iloc[similar_indices].tolist()
    top_similar_df.loc[artist] = similar_artists

top_similar_df.head()

,SimilarArtist_1,SimilarArtist_2,SimilarArtist_3,SimilarArtist_4,SimilarArtist_5,SimilarArtist_6,SimilarArtist_7,SimilarArtist_8,SimilarArtist_9,SimilarArtist_10
artist,,,,,,,,,,
massive attack,morcheeba,portishead,chris coco,terranova,leftfield,unkle,alpha,celestial,helicopter girl,solar fields
apartment 26,shining,exilia,bloodsimple,emil bulls,flaw,slitheryn,hamlet,static‐x,jonathan davis,tracktor bowling
robert miles,dzihan & kamien,terranova,kevin yost,trentemøller,alpha,goldfish,st germain,bt,john digweed,torsten stenzel
vincent gallo,kelli hand,beans,kenny larkin,dj mink,drexciya,home video,thk,elecktroids,ko-wreck technique,rubber johnny
squirrel nut zippers,eddie sauter,john pizzarelli,tyree glenn,dave mckenna,charlie barnet,cozy cole,dick mcdonough,lester young,charlie shavers,eric delaney


In [24]:
top_similar_df.loc["massive attack"].tolist()

['morcheeba',
 'portishead',
 'chris coco',
 'terranova',
 'leftfield',
 'unkle',
 'alpha',
 'celestial',
 'helicopter girl',
 'solar fields']

In [18]:
input_artist = "Coldplay"
recommendations = pd.DataFrame(df_mb3.nlargest(11,input_artist)['artist'])
recommendations = recommendations[recommendations['artist']!=input_artist]
print(recommendations)

KeyError: 'Coldplay'